Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Aseem Sachdeva"

---

# Information Visualization I 
## School of Information, University of Michigan

## Week 1: 
- Domain identification vs Abstract Task extraction
- Pandas Review 

## Assignment Overview
### The objectives for this week are for you to:

- Review, reflect, and apply the concepts of Domain Tasks and Abstract Tasks. Specifically, given a real context, identify the expert's goals and then abstract the visualization tasks. 

<img src="assets/domain-abstraction.png" alt="Drawing" style="width: 500px;"/>

- Review and evaluate the domain of [Pandas](https://pandas.pydata.org/) as a tool for reading, manipulating, and analyzing datasets in Python.

### The total score of this assignment will be 100 points consisting of:
- Case study reflection: Car congestion and crash rates (20 points)
- Pandas programming exercise (80 points)


### Resources:
- We're going to be recreating parts of this article by [CMAP](https://www.cmap.illinois.gov/) available [online](https://www.cmap.illinois.gov/updates/all/-/asset_publisher/UIMfSLnFfMB6/content/crash-scans-show-relationship-between-congestion-and-crash-rates) (CMAP, 2016)  
- We'll need the datasets from the city of Chicago. We have downloaded a subset to the local folder [/assets](assets/)
    - If you're curious, the original dataset can be found on [Chicago Data Portal](https://data.cityofchicago.org/)
        - [Chicago Traffic Tracker - Historical Congestion Estimates by Segment - 2011-2018](https://data.cityofchicago.org/Transportation/Chicago-Traffic-Tracker-Historical-Congestion-Esti/77hq-huss)
        - [Traffic Crashes - Crashes](https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if)
- Altair
    - We will use a python library called [Altair](https://altair-viz.github.io/) for the visualizations. Don't worry about understanding this code. You will only need to prepare the data for the visualization in Pandas. If you do it correctly, our code will produce the visualization for you.

## Part 1. Domain identification vs Abstract Task extraction (20 points)
Read the following article by CMAP [Crash scans show the relationship between congestion and crash rates](https://www.cmap.illinois.gov/updates/all/-/asset_publisher/UIMfSLnFfMB6/content/crash-scans-show-relationship-between-congestion-and-crash-rates) and answer the following questions:


### 1.1 Briefly describe who you think performed this analysis. What is their expertise? What is their goal for the article? Give 3 examples of domain tasks featured in the article. (10 points)

The fact that the above article is hosted under the Chicaco Metropolitan Agency for Planning's(CMAP) governmental web domain name highly suggests that the analyst who authored the article is affiliated with the government of illinois through the aforementioned organization. The analayst can feasibly tout a degree of expertise in highway/roadway planning on account of the relative ease and accuracy with which they utilize domain specific vocabulary. Three examples of domain-specific tasks discussed in the article are:

1. What is the relationship between highway congestion and crash rates?
2. Do arterial roadways witness crashes at a different rate than highways?
3. Do highways with excess tollways/merge locations witness crashes at a different rate than highways with a more free design?

### 1.2  For each domain task describe the abstract task (10 points)

1. Find the correlation between variable A and variable B
2. Find the statistical difference between population A and population B
3. Find the statistical difference between population A and population B

## Part 2. Pandas programming exercise (80 points)
We have provided some code to create visualizations based on these two datasets:
1. [Historic Congestion](assets/Pulaski.small.csv.gz) 
2. [Traffic Crashes](assets/Traffic.Crashes.csv.gz)

Complete each assignment function and run each cell to generate the final visualizations


In [2]:
import pandas as pd
import numpy as np
import altair as alt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# enable correct rendering
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [4]:
# uses intermediate json files to speed things up
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

### PART A: Historic Congestion ( 55 points)
For parts 2.1 to 2.5 we will use the Historic Congestion dataset. This dataset contains measures of speed for different segments. For this subsample, the available measures are limited to traffic on Pulaski Road in 2018.

### 2.1 Read and resample (15 points)
Complete the `read_csv` and `get_group_first_row` functions.
Since our dataset is large we want to only grab one measurement per hour for each segment. To do this, we will resample by selecting the first measure for each month, day, hour on each segment. Complete the `get_group_first_row` function to achieve this. Note that the file we are loading is compressed--depending on how you load the file, this may or may not make a difference ([you'll want to look at the API documents](https://pandas.pydata.org/pandas-docs/stable/reference/index.html)).

In [5]:
def read_csv(filename):
    """Read the csv file from filename (uncompress 'gz' if needed)
    return the dataframe resulting from reading the columns
    """
    dataframe = pd.read_csv(filename)
    return dataframe
    
    #raise NotImplementedError()

In [6]:
# Save the congestion dataframe on hist_con
hist_con = read_csv('assets/Pulaski.small.csv.gz')
print(hist_con.shape)
assert hist_con.shape == (3195450, 10)
assert list(hist_con.columns) == ['TIME','SEGMENT_ID','SPEED','STREET','DIRECTION','FROM_STREET','TO_STREET',
                                  'HOUR','DAY_OF_WEEK','MONTH']

(3195450, 10)


In [7]:
def get_group_first_row(df, grouping_columns):
    """Group rows for the grouping columns and return the first row belonging to each group
    (you can look at first() for reference). We'll write this function to be more general in case
    we want to use it for a different resample.
    return a dataframe without a hierarchical index (use default index)
    
    See the example link below if you want a better sense of what this should return
    """
    dataframe = df
    dataframe_2 = dataframe.groupby(grouping_columns).first()
    dataframe_2 = dataframe_2.reset_index(level=grouping_columns)
    return dataframe_2
    #raise NotImplementedError()

In [8]:
# test your code, we want segment_rows to be resampled version of hist_con where we've grouped by the
# properties month, day_of_week, hour, and segment_id and returned the first measure of each group
segment_rows = get_group_first_row(hist_con, ['MONTH','DAY_OF_WEEK', 'HOUR', 'SEGMENT_ID'])


The table should look something like [this](assets/segment_rows.png)

In [9]:
#hidden tests are within this cell

### 2.2 Basic Bar Chart Visualization (10 points)
We want to create a visualization for the *average speed* of each segment (across all the samples). To do this, we're going to want to group by each segment and calculate the average speed on each. Complete this code on the `average_speed_per_segment` function.

In [10]:
def average_speed_per_segment(df):
    """Group rows by SEGMENT_ID and calculate the mean of each
    return a series where the index is the segment id and each value is the average speed per segment
    """
    dataframe = df
    dataframe = dataframe.groupby('SEGMENT_ID').mean()
    dataframe = dataframe.drop(['MONTH', 'DAY_OF_WEEK', 'HOUR'], axis=1)
    return dataframe

    #raise NotImplementedError()

In [11]:
average_speed_per_segment(segment_rows)

,SPEED
SEGMENT_ID,
19,12.251926
20,15.274452
21,12.141079
22,12.346769
23,12.716657
24,14.882632
25,13.076467
26,15.965027
27,21.769413


In [12]:
# calculate the average speed per segment
average_speed = average_speed_per_segment(segment_rows)

# create labels for the visualization
labels = average_speed.index.astype(str)

# grab the values from the table
values = pd.DataFrame(average_speed).reset_index()

# create a chart
base = alt.Chart(values)

# we're going to "encode" the variables, more on this next assignment
encoding = base.encode(
    x= alt.X(
            'SEGMENT_ID:Q',
            title='Segment ID',
            scale=alt.Scale(zero=False)
    ),
    y=alt.Y(
            'sum(SPEED):Q',
            title='Speed Average MPH'
    ),
)

# we're going to use a bar chart and set various parameters (like bar size and title) to make it readable
encoding.mark_bar(size=7).properties(title='Average Speed per Segment',height=300, width=900)

alt.Chart(...)

The table should look something like [this](assets/average_speed.png)

In [13]:
#hidden tests are within this cell

### 2.3 Create a basic pivot table (10 points)
For the next visualization, we need a more complex transformation that will allow us to see the average speed for each month. To do this, we will create a pivot table where the index is the month, and each column is a segment id. We will put the average speed in the cells. From the table, we'll be able to find the month (by index)--giving us the row, and pick the column corresponding to the segment we care about.

Complete the `create_pivot_table` function for this

In [14]:
def create_pivot_table (df):
    """return a pivot table where:
    each row i is a month
    each column j is a segment id
    each cell value is the average speed for the month i in the segment j
    """
    dataframe = df
    table = pd.pivot_table(dataframe, values='SPEED', index=['MONTH'],
                    columns=['SEGMENT_ID'], aggfunc=np.mean)
    return table
    
    #raise NotImplementedError()

In [15]:
# run the code and see what's in the table
pivot_table = create_pivot_table(segment_rows)
pivot_table

SEGMENT_ID,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97
MONTH,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,6.857143,16.142857,13.571429,19.571429,18.285714,15.857143,11.285714,10.142857,25.000000,20.571429,20.000000,23.857143,10.571429,4.857143,9.857143,15.857143,19.142857,27.142857,20.714286,12.714286,19.857143,17.714286,18.142857,19.000000,21.000000,24.285714,12.000000,8.428571,9.142857,12.285714,9.714286,18.428571,16.285714,20.142857,20.571429,16.714286,10.714286,23.428571,12.571429,20.142857,22.428571,17.285714,19.857143,22.714286,26.142857,19.142857,21.714286,19.285714,9.714286,12.571429,7.000000,9.142857,16.428571,12.142857,17.428571,11.571429,10.142857,15.857143,21.857143,15.714286,13.428571,20.428571,16.714286,13.857143,22.428571,25.714286,16.142857,18.428571,17.000000,14.714286,19.000000,17.857143,20.857143,12.000000,16.857143,14.857143,22.285714,17.857143
3,10.773810,14.863095,11.696429,11.815476,13.583333,16.244048,12.398810,15.529762,21.779762,12.422619,12.059524,13.107143,14.708333,13.547619,13.071429,12.208333,13.791667,16.130952,13.916667,13.619048,15.714286,14.047619,15.857143,13.809524,16.041667,20.166667,17.232143,13.214286,13.041667,15.678571,14.136905,12.666667,12.125000,15.285714,15.226190,14.690476,15.446429,20.238095,17.833333,16.517857,19.136905,16.053571,14.732143,13.732143,12.952381,11.571429,13.958333,21.351190,14.410714,15.095238,14.190476,12.142857,11.988095,8.577381,11.785714,14.916667,17.666667,14.773810,15.952381,15.351190,14.625000,15.279762,13.035714,10.089286,19.595238,16.928571,15.851190,15.833333,15.130952,16.470238,17.744048,16.095238,18.095238,13.994048,15.875000,14.761905,20.761905,17.625000
4,11.744048,14.958333,11.791667,12.071429,13.208333,16.779762,14.136905,18.339286,22.232143,11.589286,11.505952,11.470238,11.922619,12.785714,12.833333,12.148810,14.285714,17.303571,14.404762,13.934524,16.232143,14.333333,16.273810,16.047619,18.267857,19.827381,17.476190,13.726190,14.797619,16.404762,14.113095,12.970238,12.559524,14.500000,14.613095,16.101190,15.238095,23.678571,19.023810,16.244048,17.648810,13.809524,11.934524,13.535714,12.714286,11.535714,15.357143,22.172619,14.750000,15.244048,12.767857,11.720238,12.017857,9.833333,11.488095,13.863095,17.398810,13.821429,16.244048,14.738095,14.744048,15.940476,12.809524,11.011905,19.375000,15.428571,14.422619,15.476190,14.958333,14.642857,17.702381,15.386905,18.488095,14.250000,14.803571,16.535714,23.005952,19.803571
5,11.357143,14.738095,11.369048,11.916667,12.023810,13.220238,11.505952,15.095238,22.857143,11.892857,11.190476,10.440476,9.166667,12.130952,12.059524,12.428571,13.773810,16.964286,12.851190,12.946429,14.732143,14.244048,15.690476,14.005952,15.940476,19.369048,16.255952,11.684524,12.375000,13.857143,13.077381,11.875000,13.220238,15.910714,13.708333,13.755952,13.505952,18.696429,17.178571,14.440476,17.559524,12.125000,10.440476,10.803571,10.577381,11.273810,15.601190,21.827381,13.696429,12.916667,10.904762,10.970238,11.636905,8.184524,10.863095,13.071429,14.678571,13.089286,14.517857,13.613095,13.517857,14.601190,11.255952,9.797619,18.517857,15.910714,13.886905,15.892857,14.154762,12.553571,16.184524,15.130952,17.952381,12.607143,12.976190,14.065476,20.071429,17.190476
6,11.630952,14.583333,13.011905,12.279762,12.428571,14.678571,12.690476,15.244048,22.309524,12.619048,11.863095,12.136905,10.815476,12.154762,12.821429,11.166667,12.065476,15.077381,13.148810,13.130952,15.827381,14.904762,16.208333,16.226190,19.547619,22.577381,18.827381,14.107143,15.428571,17.101190,15.095238,12.327381,12.107143,13.892857,12.976190,15.559524,14.523810,19.214286,16.875000,14.910714,18.190476,15.136905,13.845238,13.636905,12.523810,11.726190,14.696429,20.779762,13.755952,15.005952,12.630952,11.136905,13.005952,8.654762,11.779762,16.815

The table should look something like [this](assets/pivot_table.png)

In [16]:
# we're going to implement a transformation to put the pivot table into a 'long form' because it
# is easier to specify the visualization. You can print out hm_pivot_table to see what it looks like
hm_pivot_table = pivot_table.copy().unstack().reset_index()
hm_pivot_table['SPEED'] = hm_pivot_table[0]
hm_pivot_table.drop(0,axis=1,inplace=True)

# create the visualization. We're going to use rectangles (a heat map of sorts). We'll use the segment_id to
# figure out the horizontal placement (x), the month as the vertical (y) and use color to encode the speed.
encoding = alt.Chart(hm_pivot_table).mark_rect().encode(
    x='SEGMENT_ID:O',
    y='MONTH:O',
    color='SPEED:Q'
)

encoding.properties(title='Average Speed per Segment per Month',height=300, width=800)

alt.Chart(...)

In [17]:
# test function
pivot_table = create_pivot_table(segment_rows)
# check that the rows are months and columns are segments
assert pivot_table.shape == (11, 78), "Problem 2.3, first test"
# check that the value is the average
assert int(pivot_table.loc[2,19]) == 6, "Problem 2.3, second test"
assert int(pivot_table.loc[3,19]) == 10, "Problem 2.3, third test"

### 2.4 Sorting, Transforming, and Filtering (20 points)
Without telling you too much about the visualization we want to create next (that's part of the bonus below), we need to get the data into a form we can use. 
- We're going to need to sort the dataframe by one or more columns (this is the `sort` function). 
- We'll want to create a derivative column that is the time of the measurement rounded to the nearest hour (`time_to_hours`)
- We need to "facet" the data into groups to generate different visualizations. 
- We need a function that selects part of the dataframe that matches a specific characteristic (`filter_orientation`)
- Grab a specific column from the dataframe (`select_column`)

In [18]:
def sort(df, sorting_columns):
    """Sort the rows by the columns
    return the sorted dataframe
    """
    dataframe = df
    dataframe = dataframe.sort_values(sorting_columns)
    return dataframe
    # YOUR CODE HERE
    #raise NotImplementedError()

In [19]:
segment_rows = sort(segment_rows, ['SEGMENT_ID'])

In [20]:
#hidden tests are within this cell

In [21]:
def time_to_hours(df):
    """ Add a column (called TIME_HOURS) based on the data in the TIME column and rounded up 
    the value to the nearest hour.  For example, if the original TIME row said: 
    ‘02/28/2018 05:40:00 PM’ we want ‘2018-02-28 18:00:00’  
    (the change is that 5:40pm was rounded up to 6:00pm and the TIME_HOUR column is 
    actually a proper datetime and not a string).
    """
    dataframe = df
    dataframe['TIME_HOURS'] = pd.to_datetime(dataframe['TIME']).dt.round('H')
    #dataframe['TIME_HOURS'] = dataframe['TIME_HOURS'].dt.round('H')
    return dataframe
    #raise NotImplementedError()

In [22]:
segment_rows = time_to_hours(segment_rows)


In [23]:
#hidden tests are within this cell

In [24]:
def filter_orientation(df, traffic_orientation):
    """ Filter the rows according to the traffic orientation
    return a df that is a subset of the original with the desired orientation
    """
    dataframe = df
    dataframe_subset = dataframe[dataframe['DIRECTION'] == traffic_orientation]
    return dataframe_subset
    #raise NotImplementedError()

In [25]:
sb = filter_orientation(segment_rows, 'SB')
nb = filter_orientation(segment_rows, 'NB')


The sb table should look something like [this](assets/sb.png)

In [26]:
#hidden tests are within this cell

In [27]:
def select_column(df, column_name):
    """ Select a column from the df
    return a series with the desired column
    """
    dataframe = df
    dataframe_selected_column = dataframe[column_name]
    return dataframe_selected_column
    #raise NotImplementedError()

In [28]:
#hidden tests are within this cell

In [29]:
# we're going to remove speeds of -1 (no data)
sb = sb[sb.SPEED > -1]
nb = nb[nb.SPEED > -1]

In [30]:

alt.data_transformers.disable_max_rows()
alt.Chart(sb.append(nb)).mark_rect().encode(
    x='month(TIME_HOURS):T',
    y='FROM_STREET:N',
    color='mean(SPEED):Q',
    facet='DIRECTION:N'
).properties(
    width=300,
    height=400
)


alt.Chart(...)

### 2.5 (Bonus)  Traffic heatmap visualization (up to 2 points)
Looking at the visualization above (the one showing Northbound versus Southbound facets), what domain/abstract tasks are fulfilled by this visualization? List at least one domain task and the corresponding abstract task.

Domain Task: Is there a relationship between time and traffic congestion on northbound lanes and southbound lanes, and is there a significant difference between both populations?

Abstract Tasks: (1) Find the correlation between variable A and variable B
                (2) Find the statistical difference between population A and population B

### PART B: Crashes (25 points)
For parts 2.6 and 2.7 we will use the Crashes dataset. This dataset contains crash entries recording the time of the accident, the street, and the street number where the accident occurred. You will work with accidents recorded on Pulaski Road


In [31]:
crashes = read_csv('assets/Traffic.Crashes.csv.gz')
crashes_pulaski = crashes[crashes.STREET_NAME == 'PULASKI RD']

In [32]:
crashes_pulaski['STREET_NO'].max()

11480

### 2.6 Calculate summary statistics for grouped streets (15 points)
- Group the streets every 300 units (street numbers). Hint: You can use the `pd.cut` function
- Calculate the number of accidents (count rows) and the total of injuries (sum injuries total) for each of these 300-chunk road segments. Do this *for each direction*.

Complete bin_crashes and calculate_group_aggregates functions for this

In [33]:
def bin_crashes(df):
    """ Assign each crash instance a category (bin) every 300 house number units starting from 0
    Return a new dataframe with a column called BIN where each value is the start of the bin
    i.e. 0 is the label for records with street number n, such that 1 <= n <= 300
    300 is the label for records with with n at 301 <= n <= 600, and so on.
    """
    bin_values = np.arange(0,12000,300)
    df['BIN'] = pd.cut(df['STREET_NO'],bin_values,labels=bin_values[0:-1])
    return df
    #raise NotImplementedError()

In [34]:
binned_df = bin_crashes(crashes_pulaski)

In [35]:
binned_df.head()

,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,PHOTOS_TAKEN_I,STATEMENTS_TAKEN_I,DOORING_I,WORK_ZONE_I,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,BIN
6,JC100005,NaN,12/31/2018 11:45:00 PM,25,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,UNKNOWN,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,4.0,STRAIGHT AND LEVEL,UNKNOWN,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,Y,NaN,Y,"OVER $1,500",12/31/2018 11:55:00 PM,IMPROPER TURNING/NO SIGNAL,IMPROPER LANE USAGE,2200,S,PULASKI RD,1013,NaN,NaN,NaN,N,NaN,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,0.0,23,2,12,41.851521,-87.724905,POINT (-87.72490478675 41.851521442331),2100
51,JB574321,NaN,12/31/2018 09:20:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,4.0,STRAIGHT AND LEVEL,WET,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,NaN,NaN,Y,"OVER $1,500",12/31/2018 09:24:00 PM,FAILING TO REDUCE SPEED TO AVOID CRASH,NOT APPLICABLE,4699,S,PULASKI RD,821,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,21,2,12,41.807977,-87.723443,POINT (-87.723442855227 41.807977202659),4500
126,JB574112,NaN,12/31/2018 05:30:00 PM,30,NO CONTROLS,NO CONTROLS,RAIN,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,PARKING LOT,0.0,STRAIGHT AND LEVEL,WET,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,NaN,Y,NaN,"$501 - $1,500",12/31/2018 06:03:00 PM,IMPROPER BACKING,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,5139,S,PULASKI RD,822,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,17,2,12,41.799320,-87.723191,POINT (-87.72319071101 41.799320431921),5100
133,JC111739,NaN,12/31/2018 05:30:00 PM,30,NO CONTROLS,NO CONTROLS,RAIN,DUSK,REAR END,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,WET,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"$501 - $1,500",01/10/2019 02:42:00 PM,NOT APPLICABLE,NOT APPLICABLE,3900,S,PULASKI RD,815,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,4.0,0.0,17,2,12,41.822564,-87.724187,POINT (-87.724187479133 41.822563835982),3600
240,JB573809,NaN,12/31/2018 01:30:00 PM,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,TURNING,DIVIDED - W/MEDIAN (NOT RAISED),4.0,STRAIGHT AND LEVEL,WET,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,NaN,NaN,NaN,"OVER $1,500",12/31/2018 01:33:00 PM,IMPROPER LANE USAGE,UNABLE TO DETERMINE,5301,S,PULASKI RD,822,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,6.0,0.0,13,2,12,41.796759,-87.723116,POINT (-87.723115903434 41.79675909896),5100


A sample of the relevant columns from the table would look something like [this](assets/binned_df.png). We can also create a histogram of street numbers to see which are the most prevalent. It should look something like [this](assets/street_no.png).

In [36]:
# create this vis
alt.Chart(binned_df).mark_bar().encode(
    alt.X('BIN'),
    alt.Y('count()')
)

alt.Chart(...)

In [37]:
#hidden tests are within this cell

In [38]:
def calculate_group_aggregates(df):
    """ 
    Return a df with the count of accidents in a 'ACCIDENT_COUNT' column and 'INJURIES_SUM'
    """
    dataframe = df
    grouping_columns = ['BIN', 'STREET_DIRECTION']
    dataframe = dataframe.groupby(grouping_columns)['INJURIES_TOTAL'].agg({'ACCIDENT_COUNT': 'count', 'INJURIES_SUM': 'sum'})
    dataframe = dataframe.reset_index(level=grouping_columns)
    
    return dataframe
    #raise NotImplementedError()

In [39]:
aggregates = calculate_group_aggregates(binned_df)
aggregates.head(5)


,BIN,STREET_DIRECTION,ACCIDENT_COUNT,INJURIES_SUM
0,0,N,52.0,18.0
1,0,S,44.0,17.0
2,300,N,37.0,19.0
3,300,S,49.0,17.0
4,600,N,69.0,19.0


The table should look something like [this](assets/aggregates.png)

In [40]:
#hidden tests are within this cell

Just for fun, here's a plot of injuries in the North and South directions based on bin. This may also help you debug your code. Depending on whether you removed N/A or if you hardcoded things, you may see slight differences. Here's what it might  [look like](assets/direction_injuries.png)

In [41]:
alt.Chart(aggregates).mark_point().encode(
    alt.Color('STREET_DIRECTION'),
    alt.X('BIN'),
    alt.Y('INJURIES_SUM')
)

alt.Chart(...)

### 2.7 Sort the street ranges (10 points)
- Sort the dataframe so North streets are in descending order and South streets are in ascending order
- You are provided with a 'sort' arrray that contains this desired order. Use a categorical (pd.Categorial) column to order the dataframe according to this array.

In [42]:
crashed_range = list(range(0, crashes_pulaski.STREET_NO.max()+1000, 300))
sort = ['N ' + str(s) for s in crashed_range[::-1]] + ['S ' + str(s) for s in crashed_range]
def categorical_sorting(df, sort):
    """ Create a column called ORDER_LABEL that contains a concatenation of the street direction and the street range
    Set the sort order of this column to the provided sort array (the elements of this column should be in the same order
    of the array)
    Sort the dataframe by this column
    """
    dataframe = df
    dataframe['BIN'] = dataframe['BIN'].astype(str)
    dataframe['STREET_DIRECTION'] = dataframe['STREET_DIRECTION'].astype(str)
    dataframe['ORDER_LABEL'] = dataframe['STREET_DIRECTION'] + ' ' +  dataframe['BIN']
    dataframe['ORDER_LABEL'] = pd.Categorical(dataframe['ORDER_LABEL'], 
                        categories=sort,
                        ordered=True)
                                                        
    dataframe.sort_values('ORDER_LABEL', inplace=True)                                                 

    return dataframe
    #raise NotImplementedError()

In [43]:
sorted_groups = categorical_sorting(aggregates, sort)


The table should look something like [this](assets/sorted_groups.png)

In [44]:
#hidden tests are within this cell

Again, just for kicks, let's see where injuries happen. We're going to color bars by the bin and preserve our ascending/descending visualization. We can probably imagine other (better) ways to visualize this data, but this may be useful for you to debug. The visualization should look something like [this](assets/order_injuries.png)

In [45]:
alt.Chart(sorted_groups).mark_bar().encode(
    alt.X('ORDER_LABEL:O', sort=sort),
    alt.Y('INJURIES_SUM:Q'),
    alt.Color('BIN:Q')
).properties(
    width=400
)

alt.Chart(...)

Ok, let's actually make a useful visualization using some of the dataframes we've created. As a bonus, we're going to ask you what you would use this for.

In [46]:
# to make the kind of chart we are interested in we're going to build it out of three different charts and
# put them together at the end

# this is going to be the left chart
bar_sorted_groups = sorted_groups[['ACCIDENT_COUNT','INJURIES_SUM']].unstack().reset_index() \
    .rename({'level_0':'TYPE','level_1':'SPEED',0:'COUNT'},axis=1)

a = alt.Chart(bar_sorted_groups).mark_bar().transform_filter(alt.datum.TYPE == 'ACCIDENT_COUNT').encode(
    x=alt.X('COUNT:Q',sort='descending'),
    y=alt.Y('SPEED:O',axis=None),
    color=alt.Color('TYPE:N', 
                    legend=None,
                    scale=alt.Scale(domain=['ACCIDENT_COUNT', 'INJURIES_SUM'],
                                    range=['blue', 'orange']))
).properties(
    title='ACCIDENT_COUNT',
    width=300,
    height=600
)

# middle "chart" which actually won't be a chart, just a bunch of labels
b = alt.Chart(bar_sorted_groups).mark_bar().transform_filter(alt.datum.TYPE == 'ACCIDENT_COUNT').encode(
    y=alt.Y('SPEED:O', axis=None),
    text=alt.Text('SPEED:Q')
).mark_text().properties(title='SPEED',
                         width=20,
                         height=600)

# and the right most chart
c = alt.Chart(bar_sorted_groups).mark_bar().transform_filter(alt.datum.TYPE == 'INJURIES_SUM').encode(
    x='COUNT:Q',
    y=alt.Y('SPEED:O',axis=None),
    color=alt.Color('TYPE:N', 
                    legend=None,
                    scale=alt.Scale(domain=['ACCIDENT_COUNT', 'INJURIES_SUM'],
                                    range=['blue', 'orange']))
).properties(
    title='INJURIES_SUM',
    width=300,
    height=600
)

# put them all together 

a | b | c


alt.HConcatChart(...)

## 2.8 (Bonus) Accident barchart visualization (up to 2 points)
Looking at the visualization we generated above (part 2.7), what domain/abstract tasks are fulfilled by this visualization? List at least one domain task and the corresponding abstract task. 

Domain Task: What is the relationship between speed and accident count, and speed and total injuries?

Abstract Task: Find the correlation between variable A and variable B